## Задание 1.
#### Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [89]:
KEYWORDS = ['python', 'парсинг', 'vpn']

In [82]:
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text)

In [83]:
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/city_scraper/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/8f8/d6c/328/8f8d6c328ec3a513103b6757c2c21610.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">city_scraper</span>
 </a>
 <span class="post__time">сегодня в 10:24</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/549604/">Зимний Нижний Тагил: город первой железной дороги в стране</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/Ecology/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Экология'); }" rel="nofollow" title="Вы не подписаны на этот ха

In [90]:
search = pd.DataFrame()

#данный цикл ищет только на текущей странице сайта, а не по всему сайту
for post in posts:
    title = post.find('a', class_='post__title_link').text.lower() #заголовок в нижнем регистре
    abstract = post.find('div', class_='post__text').text.lower() #резюме поста в нижнем регистре
    
    for keyword in KEYWORDS:
        if keyword in title or keyword in abstract:
            link = post.find('a', class_='post__title_link').attrs.get('href') #выделяем ссылку на пост из атрибутов тега
            #получаю дату из атрибута 'data-time_published', переходя по ссылке непосредстенно в саму статью
            date = pd.to_datetime(BeautifulSoup(requests.get(link).text).find('span', class_='post__time').get('data-time_published'))
            row = {'date': date, 'title': title, 'link': link}
            search = pd.concat([search, pd.DataFrame([row])]) 
search

,date,title,link
0,2021-03-30 02:21:00+00:00,«живые» обои для компьютера на python,https://habr.com/ru/post/549638/
0,2021-03-29 17:40:00+00:00,анимация волновой функции частицы шрёдингера (...,https://habr.com/ru/company/skillfactory/blog/...


#### Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [91]:
search_in = pd.DataFrame()

#данный цикл ищет только на текущей странице сайта, а не по всему сайту
for post in posts:
    title = post.find('a', class_='post__title_link').text.lower() #заголовок в нижнем регистре
    abstract = post.find('div', class_='post__text').text.lower() #резюме поста в нижнем регистре
    link = post.find('a', class_='post__title_link').attrs.get('href') #выделяем ссылку на пост из атрибутов тега
    soup_post = BeautifulSoup(requests.get(link).text)
    text = soup_post.find('div', class_='post__body').text.lower()
    
    for keyword in KEYWORDS:
        if keyword in title or keyword in abstract or keyword in text:
            date = pd.to_datetime(soup_post.find('span', class_='post__time').get('data-time_published'))
            row = {'date': date, 'title': title, 'link': link, 'text': text}
            search_in = pd.concat([search_in, pd.DataFrame([row])])

search_in

,date,title,link,text
0,2021-03-30 06:30:00+00:00,погружение в serverless. по следам протокола s3,https://habr.com/ru/post/547346/,\nпродолжаем беседовать с разработчиками экоси...
0,2021-03-30 06:00:00+00:00,не только лишь удаленка: как атаковали киберпр...,https://habr.com/ru/company/solarsecurity/blog...,\nситуация с иб в 2020-м напоминала картины бо...
0,2021-03-30 02:21:00+00:00,«живые» обои для компьютера на python,https://habr.com/ru/post/549638/,"\nне ругайтесь сильно, программирование - лишь..."
0,2021-03-29 20:09:00+00:00,как просто заблокировать сайты в веб- приложен...,https://habr.com/ru/company/hetmansoftware/blo...,\nогромный объем информации окружает пользоват...
0,2021-03-29 20:08:00+00:00,мультитул для управления хранилищем данных — к...,https://habr.com/ru/company/wheely/blog/549614/,\nуже более двух лет data build tool активно и...
0,2021-03-29 17:40:00+00:00,анимация волновой функции частицы шрёдингера (...,https://habr.com/ru/company/skillfactory/blog/...,\nдвойственная природа материи — широко извест...


## Задание 2.
#### Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [2]:
EMAIL = ['xxx@x.ru', 'zzz@ya.ru']

#сайт, к которому на самом деле осуществляется запрос:
URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

#методом тыка определяю, какие параметры нужно добавить в headers
headers = {
    'Content-Type': 'application/json;charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'vaar-header-captcha-response-token': '03AGdBq24BbNS_aj-HmJQVdI3SumvAdGdlpaih52OTqilfAqxp4VkiPhVptAgwke0e0GCdfGnXWw2tMyR8EYu3VSOJ2xg8LbaYdQf8dhjkXm2-rHXMCQps0qGMzynT0IKFTBbj23NhAuF76HLeNPr6sLmvUpW8JsP7LjHMLJIPSlW8Ll_1iErvNUDOWoqUlrNTCYDJ8k69ZSd6A31lUYgaMZAXbOmhJk_GzpT-fYje3WwIkgzzsPJ2tMDSUgddBrwECtjHg4QdA7cq8UBUNJdkwDOxN7hE_WK9H0qKdl-gsiWLroX4iFfdHkJyksN8-JkHIO7-wGI4KJQm6J0rYatDjKGQkjCRPMwsrHF8crvZiakfEtMroUY0ueHuCDlIZTte5OEhs_BM48sg8eqBhZG7jZePr6X51n_KaSUB0gXfcGJAjsSnqOmReD_wx5id0dcLwyqihq6pqlX9',
    'Vaar-Version': '0'
}

In [ ]:
#import time
#time.sleep(0.2)

In [3]:
leaks = pd.DataFrame()

req = requests.post(URL, json={'emailAddresses': EMAIL}, headers=headers)
leaks_json = req.json()

for mail in EMAIL:
    breaches_ids = leaks_json['summary'][mail]['breaches'] #выделяем из словаря id утечек, получаем список для каждой почты из списка
    for i in breaches_ids:
        leak_date = pd.to_datetime(leaks_json['breaches'][str(i)]['publishDate'])
        leak_source = leaks_json['breaches'][str(i)]['site']
        leak_desc = leaks_json['breaches'][str(i)]['description']
        row = {'email': mail, 'leak_date': leak_date, 'leak_source': leak_source, 'leak_description': leak_desc}
        leaks = pd.concat([leaks, pd.DataFrame([row])])
        
leaks

,email,leak_date,leak_source,leak_description
0,xxx@x.ru,2017-02-14 00:00:00+00:00,parapa.mail.ru,"In July and August 2016, two criminals execute..."
0,xxx@x.ru,2016-10-29 00:00:00+00:00,vk.com,Popular Russian social networking platform VKo...
0,xxx@x.ru,2016-10-21 00:00:00+00:00,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,xxx@x.ru,2017-02-14 00:00:00+00:00,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,xxx@x.ru,2017-01-31 00:00:00+00:00,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
0,xxx@x.ru,2016-10-23 00:00:00+00:00,imesh.com,"In June 2016, a cache of over 51 million user ..."
0,zzz@ya.ru,2020-05-21 00:00:00+00:00,vk.com,"At some time in 2020, the Russian social netwo..."
0,zzz@ya.ru,2016-11-01 00:00:00+00:00,qip.ru,"In 2011, Russian instant messaging service pro..."
0,zzz@ya.ru,2020-12-10 00:00:00+00:00,typhoonclicks.info,"In November 2020, a collection of over 23,000 ..."
0,zzz@ya.ru,2017-11-04 00:00:00+00:00,myheritage.com,"In October 2017, a customer database belonging..."
